In [21]:
import json
import re
import spacy
#import nltk
import unicodedata
#from nltk.tokenize import word_tokenize
from spacy.tokenizer import Tokenizer
from tqdm import tqdm

In [2]:
#nltk.download('averaged_perceptron_tagger')
#nltk.download('punkt')

In [22]:
nlp = spacy.load("en_core_web_sm")
#nlp.tokenizer = Tokenizer(nlp.vocab, token_match=re.compile(r'\S+').match)

In [ ]:
pwd

In [ ]:
answers = []

for i in range(5):
    with open('data/kvqa/kvqa/trainval_ans2label_{}.json'.format(i)) as f:
        answers += list(json.load(f).keys())
answers = set(answers)

In [ ]:
len(answers)

In [ ]:
answers

In [4]:
def load_json(path):
    with open(path) as f:
        data = json.load(f)
    return data


def get_span(caption, lower=False):
    if not caption:
        return caption, []
    caption = caption.strip()
    caption = unicodedata.normalize("NFKD", caption)
    caption_ = caption
    if lower:
        caption = caption.lower()
    cap = nlp(caption)
    
    # All tokens are PROPN
    if all(token.pos_ == 'PROPN' for token in cap):
        return caption_, [[caption, 0, len(caption)]]
    
    mention_spans = []
    prev_pos = cap[0].pos_
    dummy_cap = cap[0].text
    span_start = 0
    span_end = 0
    for token in cap[1:]:
#         if token.text not in [',', '.', '!', '?'] \
#         and dummy_cap[-1] not in ['(', ' ', '\u2009', '\xa0']:
        if token.text not in ['  ', ' ', '']:
            dummy_cap += ' '
        if prev_pos != 'PROPN' and token.pos_ == 'PROPN':
            span_start = len(dummy_cap)
        elif prev_pos == 'PROPN' and token.pos_ != 'PROPN': 
            span_end = len(dummy_cap) - 1
            #assert dummy_cap[span_start: span_end] == caption[span_start: span_end], (span_start, span_end, dummy_cap, caption)
            mention_spans.append([dummy_cap[span_start: span_end], span_start, span_end])

        dummy_cap += token.text

        prev_pos = token.pos_
#     mention_spans_ = []
#     for ans in answers:
#         for m in mention_spans:
#             if set(m[0].split()).intersection(set(ans.split())):
#                 mention_spans_.append(m)
        
    return caption_, mention_spans


def get_span2(caption, lower=False):
    
    tags = ['NNP', 'NNPS']
    
    if not caption:
        return caption, []
    caption = caption.strip()
    caption = unicodedata.normalize("NFKD", caption)
    caption_ = caption
    if lower:
        caption = caption.lower()
    cap = nltk.word_tokenize(caption)
    cap = nltk.pos_tag(cap)
    
    # All tokens are PROPN
    if all(token[1] in tags for token in cap):
        return caption_, [[caption, 0, len(caption)]]
    
    mention_spans = []
    prev_pos = cap[0][1]
    dummy_cap = cap[0][0]
    span_start = 0
    span_end = 0
    for token in cap[1:]:
#         if token.text not in [',', '.', '!', '?'] \
#         and dummy_cap[-1] not in ['(', ' ', '\u2009', '\xa0']:
        if token[0] not in ['  ', ' ', '']:
            dummy_cap += ' '
        if prev_pos not in tags and token[1] in tags:
            span_start = len(dummy_cap)
        elif prev_pos in tags and token[1] not in tags: 
            span_end = len(dummy_cap) - 1
            #assert dummy_cap[span_start: span_end] == caption[span_start: span_end], (span_start, span_end, dummy_cap, caption)
            mention_spans.append([dummy_cap[span_start: span_end], span_start, span_end])

        dummy_cap += token[0]
        prev_pos = token[1]
    
    # Mention span at the end
    if prev_pos in tags and token[1] in tags:
        span_end = len(dummy_cap) - 1
        mention_spans.append([dummy_cap[span_start: span_end], span_start, span_end])
        
    return caption_, mention_spans


def get_span3(caption, lower=False):
    if not caption:
        return caption, []
    caption = caption.strip()
    caption = unicodedata.normalize("NFKD", caption)
    caption_ = caption
    if lower:
        caption = caption.lower()
    cap = nlp(caption)
    
    mention_spans = [(ent.text, ent.start_char, ent.end_char) for ent in cap.ents if ent.label_ == 'PERSON']

    return caption_, mention_spans


def get_span4(caption, lower=False):
    if not caption:
        return caption, []
    caption = caption.strip()
    caption = unicodedata.normalize("NFKD", caption)
    caption_ = caption
    if lower:
        caption = caption.lower()
    cap = nlp(caption)
    
    # All tokens are PROPN
    if all(token.pos_ == 'PROPN' for token in cap):
        return caption_, [[caption, 0, len(caption)]]
    
    mention_spans = []
    prev_pos = cap[0].pos_
    dummy_cap = cap[0].text
    span_start = 0
    span_end = 0
    for token in cap[1:]:
#         if token.text not in [',', '.', '!', '?'] \
#         and dummy_cap[-1] not in ['(', ' ', '\u2009', '\xa0']:
        if token.text not in ['  ', ' ', '']:
            dummy_cap += ' '
        if prev_pos != 'PROPN' and token.pos_ == 'PROPN':
            span_start = len(dummy_cap)
        elif prev_pos == 'PROPN' and token.pos_ != 'PROPN': 
            span_end = len(dummy_cap) - 1
            #assert dummy_cap[span_start: span_end] == caption[span_start: span_end], (span_start, span_end, dummy_cap, caption)
            mention_spans.append([dummy_cap[span_start: span_end], span_start, span_end])

        dummy_cap += token.text

        prev_pos = token.pos_

    for e in [(ent.text, ent.start_char, ent.end_char) for ent in cap.ents if ent.label_ == 'PERSON']:
        if e[0] not in [m[0] for m in mention_spans]:
            mention_spans.append(e) 
        
    return caption_, mention_spans


def write_spans(read_from, save_to, span_fn):
    data = load_json(read_from)
    good = 0
    bad = 0
    really_bad = 0
    output = {}
    with open(save_to, 'w') as f:
        i = 0
        for ex in tqdm(data):
            i += 1
            img_id = ex['imgPath'].split('/')[1].split('.')[0]
            entities = ex['NamedEntities']
            caption = ex['wikiCap']
            caption, ent_spans = span_fn(caption)
            output[img_id] = {
                'wikiCap_new': caption,
                'ents_found': ent_spans
            }
            if len(entities) == len(set([es[0] for es in ent_spans])) and set(entities) == set([es[0] for es in ent_spans]):
                good += 1
                f.write('GOOD\t{}\t{}\t{}\n'.format(entities, ent_spans, caption))
            elif any([token in set([t for ent in entities for t in ent.split()]) for es in ent_spans for token in es[0].split()]):
                bad += 1
                f.write('BAD\t{}\t{}\t{}\n'.format(entities, ent_spans, caption))
            else:
                really_bad += 1
                f.write('REALLY BAD\t{}\t{}\t{}\n'.format(entities, ent_spans, caption))
                
    print('Total: {},  Good: {},  Bad: {},  Really Bad: {}'.format(i, good, bad, really_bad))
    return output
        
            
        
        

In [ ]:
train0 = load_json('data/kvqa/kvqa/train_kvqa0.json')

In [ ]:
train0[0]['imgPath']

In [ ]:
get_span3(train0[0]['wikiCap'])

In [12]:
output = write_spans('data/kvqa/kvqa/train_kvqa0.json', 'data/kvqa/kvqa/train_kvqa0_ent_spans_1.tsv')

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 17157/17157 [02:34<00:00, 111.29it/s]

Total: 17157,  Good: 1719,  Bad: 9891,  Really Bad: 5547


In [13]:
with open('data/kvqa/kvqa/train_kvqa0_ent_spans_yasu_1.json', 'w') as f:
    json.dump(output, f)

In [14]:
output = write_spans('data/kvqa/kvqa/valid_kvqa0.json', 'data/kvqa/kvqa/valid_kvqa0_ent_spans_1.tsv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4947/4947 [00:43<00:00, 114.30it/s]

Total: 4947,  Good: 500,  Bad: 2943,  Really Bad: 1504


In [15]:
with open('data/kvqa/kvqa/valid_kvqa0_ent_spans_yasu_1.json', 'w') as f:
    json.dump(output, f)

In [16]:
output = write_spans('data/kvqa/kvqa/test_kvqa0.json', 'data/kvqa/kvqa/test_kvqa0_ent_spans_1.tsv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2498/2498 [00:22<00:00, 111.83it/s]

Total: 2498,  Good: 251,  Bad: 1492,  Really Bad: 755


In [17]:
with open('data/kvqa/kvqa/test_kvqa0_ent_spans_yasu_1.json', 'w') as f:
    json.dump(output, f)

In [ ]:
get_span3(train0[0]['wikiCap'], lower=False)

In [23]:
for i in range(1, 5):
    for split in ['train', 'valid', 'test']:
        print(f'data/kvqa/kvqa/{split}_kvqa{i}.json')
        output = write_spans(
            f'data/kvqa/kvqa/{split}_kvqa{i}.json', 
            f'data/kvqa/kvqa/{split}_kvqa{i}_ent_spans_0.tsv',
            get_span
        )
        with open(f'data/kvqa/kvqa/{split}_kvqa{i}_ent_spans_yasu_0.json', 'w') as f:
            json.dump(output, f)

data/kvqa/kvqa/train_kvqa1.json


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 17105/17105 [01:03<00:00, 269.71it/s]


Total: 17105,  Good: 2707,  Bad: 10490,  Really Bad: 3908
data/kvqa/kvqa/valid_kvqa1.json


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5030/5030 [00:18<00:00, 269.16it/s]


Total: 5030,  Good: 821,  Bad: 3086,  Really Bad: 1123
data/kvqa/kvqa/test_kvqa1.json


...

Total: 17276,  Good: 2802,  Bad: 10573,  Really Bad: 3901
data/kvqa/kvqa/valid_kvqa4.json


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4894/4894 [00:18<00:00, 270.73it/s]


Total: 4894,  Good: 773,  Bad: 2985,  Really Bad: 1136
data/kvqa/kvqa/test_kvqa4.json


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2432/2432 [00:09<00:00, 270.16it/s]


Total: 2432,  Good: 387,  Bad: 1510,  Really Bad: 535


In [32]:
for i in range(1, 2):
    for split in ['train', 'valid', 'test']:
        output = write_spans(
            f'data/kvqa/kvqa/{split}_kvqa{i}.json', 
            f'data/kvqa/kvqa/{split}_kvqa{i}_ent_spans_1.tsv',
            get_span3
        )
        with open(f'data/kvqa/kvqa/{split}_kvqa{i}_ent_spans_yasu_1.json', 'w') as f:
            json.dump(output, f)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 17105/17105 [01:03<00:00, 269.68it/s]


Total: 17105,  Good: 3734,  Bad: 5663,  Really Bad: 7708


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5030/5030 [00:18<00:00, 270.31it/s]


Total: 5030,  Good: 1080,  Bad: 1730,  Really Bad: 2220


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2467/2467 [00:09<00:00, 267.96it/s]

Total: 2467,  Good: 603,  Bad: 846,  Really Bad: 1018


In [36]:
# THIS IS how the NERPER final ents were generated
for i in range(0, 5):
    for split in ['train', 'valid', 'test']:
        output = write_spans(
            f'data/kvqa/kvqa/{split}_kvqa{i}.json', 
            f'data/kvqa/kvqa/{split}_kvqa{i}_ent_spans_2.tsv',
            get_span4
        )
        with open(f'data/kvqa/kvqa/{split}_kvqa{i}_ent_spans_yasu_2.json', 'w') as f:
            json.dump(output, f)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 17157/17157 [01:04<00:00, 268.04it/s]


Total: 17157,  Good: 2689,  Bad: 10943,  Really Bad: 3525


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4947/4947 [00:18<00:00, 268.93it/s]


Total: 4947,  Good: 834,  Bad: 3185,  Really Bad: 928


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2498/2498 [00:09<00:00, 267.90it/s]


...

Total: 17276,  Good: 2760,  Bad: 11045,  Really Bad: 3471


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4894/4894 [00:18<00:00, 268.20it/s]


Total: 4894,  Good: 775,  Bad: 3115,  Really Bad: 1004


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2432/2432 [00:09<00:00, 268.21it/s]

Total: 2432,  Good: 388,  Bad: 1575,  Really Bad: 469


In [34]:
 ["Macbeth", 13, 20] in [["Macbeth", 13, 20], ["Bourchier", 0, 9]]

True

In [39]:
diego0 = load_json('data/kvqa/kvqa/kvqa_ent_spans_sept13_3corb.json')
diego1 = load_json('data/kvqa/kvqa/kvqa_ent_spans_sept13_fewkbcor.json')

In [46]:
yasu = load_json('data/kvqa/kvqa/valid_kvqa0_ent_spans_yasu_2.json')
for k, v in yasu.items():
    print(v['wikiCap_new'])
    print(v['ents_found'])
    print(diego1[k]['ents_found'])
    print()

Bourchier as Macbeth, 1910
[['Macbeth', 13, 20], ['Bourchier', 0, 9]]
[['Macbeth', 13, 20, 'Macbeth'], ['1910', 22, 26, '1910']]

Pilkington playing for Cardiff City in 2016.
[['Pilkington', 0, 10], ['Cardiff City', 23, 35]]
[['Pilkington', 0, 10, 'Pilkington'], ['Cardiff City', 23, 35, 'Cardiff City'], ['2016', 39, 43, '2016']]

Carl zu Leiningen, lithograph by Joseph Kriehuber (1833)
[['Carl zu Leiningen', 0, 17], ['Joseph Kriehuber', 34, 50]]
[['Carl zu Leiningen', 0, 17, 'Emich Carl, 2nd Prince of Leiningen'], ['Joseph Kriehuber', 33, 49, 'Joseph Kriehuber'], ['lithograph', 19, 29, 'Lithograph'], ['1833', 51, 55, '1833']]

Duke Kahanamoku with longboard, Los Angeles, 1920
[['Duke Kahanamoku', 0, 15], ['Los Angeles', 33, 44]]
[['Duke Kahanamoku', 0, 15, 'Duke Kahanamoku'], ['longboard', 21, 30, 'Longboard (skateboard)'], ['Los Angeles', 32, 43, 'Los Angeles'], ['1920', 45, 49, '1920']]

Cordell Hull as Secretary of State, from life TIME Magazine in September 1944, before his resigna

In [23]:
### ADDING NEW OKVQA SPANS 

# NERper "as is" method 
def get_span4(caption, lower=False):
    if not caption:
        return caption, []
    caption = caption.strip()
    caption = unicodedata.normalize("NFKD", caption)
    caption_ = caption
    if lower:
        caption = caption.lower()
    cap = nlp(caption)
    
    # All tokens are PROPN
    if all(token.pos_ == 'PROPN' for token in cap):
        return caption_, [[caption, 0, len(caption)]]
    
    mention_spans = []
    prev_pos = cap[0].pos_
    dummy_cap = cap[0].text
    span_start = 0
    span_end = 0
    for token in cap[1:]:
        if token.text not in ['  ', ' ', '']:
            dummy_cap += ' '
        if prev_pos != 'PROPN' and token.pos_ == 'PROPN':
            span_start = len(dummy_cap)
        elif prev_pos == 'PROPN' and token.pos_ != 'PROPN': 
            span_end = len(dummy_cap) - 1
            mention_spans.append([dummy_cap[span_start: span_end], span_start, span_end])

        dummy_cap += token.text

        prev_pos = token.pos_

    for e in [(ent.text, ent.start_char, ent.end_char) for ent in cap.ents if ent.label_ == 'PERSON']:
        if e[0] not in [m[0] for m in mention_spans]:
            mention_spans.append(e) 
        
    return caption_, mention_spans

# NEXT
# NERagro method  <-- maybe or k4 and k2p5 ,
#  kvqa_automated_metrics ( has links, noisy methods )
#  InspectResults-OKVQA

In [45]:
#import spacy
import time

#nlp = spacy.load('en_core_web_sm') 

def filter_nouns(doc, tags=['NOUN','PROPN']):
            
    output = [[tok.string.strip(), tok.idx, tok.idx + len(tok.string.strip())] for tok in doc if tok.pos_ in tags]
    return output

def filter_verbs(doc, tags=['VERB']):
    output = [[tok.string.strip(), tok.idx, tok.idx + len(tok.string.strip())] for tok in doc if tok.pos_ in tags and not tok.lemma_[0].isupper()]
    return output

def filter_noun_phrases(sentence, doc, debug = False, tags=['NOUN','PROPN']):
    res = []
    for chunk in doc.noun_chunks:          
        ebeg = chunk.start
        eend = chunk.end 
        if debug:
            print("\n\tCur chunk", chunk.string.strip(), ebeg, eend, chunk.start_char, chunk.end_char)            
            print("\t",[ (tok, tok.pos_, tok.dep_, tok.tag_) for tok in nlp(chunk.string)])           
            """
           	1. Cur chunk Which american president 0 3 0 24
            	 [(Which, 'DET', 'det', 'WDT'), (american, 'ADJ', 'amod', 'JJ'), (president, 'NOUN', 'ROOT', 'NN')]
            	add cur [['president', 15, 24]] 0 3

            2. Cur chunk the stuffed animal 7 10 49 67
            	 [(the, 'DET', 'det', 'DT'), (stuffed, 'VERB', 'amod', 'VBN'), (animal, 'NOUN', 'ROOT', 'NN')]
            	add cur [['animal', 12, 18]] 7 10 
                
            3. [                           (sporting, 'VERB', 'compound', 'VBG'), (equipment, 'NOUN', 'ROOT', 'NN')]
            """
               
        possible = [ (tok.string.strip(), tok.idx, tok.idx + len(tok.string.strip()), tok.pos_, tok.dep_, tok.tag_) for tok in nlp(chunk.string)]
        cur = []
        if len(possible) == 1 and possible[0][5] in tags:
            cur = [possible[0], possible[1], possible[2]]
        elif len(possible) > 1:
            if possible[0][4] == 'det':
                # remove determiner and then look at phrase
                possible = possible[1:]
                
            if len(possible) == 1 and possible[0][5] in tags:
                cur = [possible[0], possible[1], possible[2]]
            elif len(possible) > 1:
                cond1 = (possible[0][4] in ['amod','compound'] and possible[0][3] in ['ADJ','VERB'])
                cond2 = (possible[1][3] == 'NOUN' and possible[1][4] == 'ROOT')
                if cond1 and cond2:
                    cur = [[p[0], p[1], p[2]] for p in possible]

        # this prior filters list so it only contains NOUNS, or PROPN.. default if the cases we look for above aren't met
        if cur == []:
            cur = [[tok.string.strip(), tok.idx, tok.idx + len(tok.string.strip())] for tok in nlp(chunk.string) if tok.pos_ in tags]
        
        if cur != []:           
            if debug:
                print("\tadd cur",cur, ebeg, eend)
            all_toks = [c[0] for c in cur] 
            all_toks_str = " ".join(all_toks)   
            try:
                char_begin = sentence.index(all_toks_str)
                char_end = char_begin + len(all_toks_str)  
            except Exception as e:
                if debug:
                    print("Error with ",all_toks_str," in ",sentence)
                char_begin = sentence.index(all_toks[0])
                char_end = sentence.index(all_toks[-1]) + len(all_toks[-1])
                all_toks_str = sentence[char_begin:char_end]  
                if debug:
                    print("\tAFTER: ",all_toks_str)
            res.append([all_toks_str,char_begin, char_end]) 
    return res

def get_gen_spans(all_qs , toshow = -1):
    st = time.time()
    #all_qs = train_qs['questions'] + val_qs['questions']
    all_questions = []

    #based on word usage in questions AND answers
    ents_per_q = []
    interest_words = []
    
    for i, q in enumerate(all_qs):
        if i > toshow and toshow != -1:
            break
        #sentence = q['question']  
        sentence = q['capt']  
        #print(i, sentence)

        doc = nlp(sentence)
        nouns = filter_nouns(doc)
        verbs = filter_verbs(doc)

        debug = False

        if i < toshow: #in [7,9]:
            debug = True
            print("\n", i, sentence)
            #print(dir(doc))

        cur_ents = filter_noun_phrases(sentence, doc, debug)

        if i < toshow: #in [7,9]:
            print("\nCur ents", cur_ents)


        #add nouns
        for w in nouns:
            found = False
            for np in cur_ents:
                if w[0] in np[0]:
                    found = True
            if not found:
                #print("add noun",w)
                cur_ents.append(w)

        #add verbs
        for w in verbs:
            found = False
            for np in cur_ents:
                if w[0] in np[0]:
                    found = True
            if not found:
                #print("add verb",w)
                cur_ents.append(w)

        ents_per_q.append(cur_ents)


        i += 1

    print("Done")
    for i in range(len(ents_per_q)):
        if i < toshow:
            print(i, all_qs[i]['question'], ents_per_q[i])
            print("\t", [all_qs[i]['question'][e[1]:e[2]] for e in ents_per_q[i]])

    print("Elapsed:", time.time() - st)   #Elapsed: 442.139680147171
    return ents_per_q

In [69]:
abs_path = "../lxmert/"
OKVQA_DATA_ROOT = abs_path + 'data/okvqa/'
train_qs = json.load(open(OKVQA_DATA_ROOT + "OpenEnded_mscoco_train2014_questions_with_coco_captions.json"))
val_qs = json.load(open(OKVQA_DATA_ROOT + "OpenEnded_mscoco_val2014_questions_with_coco_captions.json"))

orig_ents = json.load(open(abs_path + "data/okvqa/okvqa_ent_spans_may06_excluded.json"))  
k4_ents = json.load(open(abs_path +  "data/okvqa/okvqa_ent_spans_sept08_cleaned.json")) 
k2p5_ents = json.load(open(abs_path +  "data/okvqa/okvqa_ent_spans_sept08_cleaned_filtered.json"))

In [35]:
print("#Questions", len(orig_ents), len(k4_ents), len(k2p5_ents))
oc = [0 for i in range(len(orig_ents))]
k4c = [0 for i in range(len(orig_ents))]
k2c = [0 for i in range(len(orig_ents))]
for i in range(len(orig_ents)):
    oc[i] = len(orig_ents[i])
    k4c[i] = len(k4_ents[i])
    k2c[i] = len(k2p5_ents[i])

def show_stats(vals):
    num_with_ents = sum([ 1 if v > 0 else 0 for v in vals])
    avg_ents = sum(vals)/len(vals)
    return [num_with_ents, round(avg_ents,3)]

print("\nQs with Ents and Avg Ents per Q")
print("Orig", show_stats(oc))
print("K4", show_stats(k4c))
print("K2p5", show_stats(k2c))

#Questions 14055 14055 14055

Qs with Ents and Avg Ents per Q
Orig [13857, 2.25]
K4 [4076, 0.329]
K2p5 [2504, 0.189]


In [37]:
st = time.time()
c = 0

all_qs = train_qs["questions"] + val_qs["questions"]
all_ents_nerper = []
for i,q in enumerate(all_qs):    
    capt_ents = get_span4(q['capt'])
    all_ents_nerper.append(capt_ents)
    if len(capt_ents[1]) > 0:# and i < 100:        
        c += 1
        """
        print("\n",i, "Question/Capt:",q)
        print("Orig ents:",orig_ents[i])
        print("4k ents:",k4_ents[i])
        print("2.5K ents:",k2p5_ents[i])
        print("Capt ents:",get_span4(q['capt']))
        """
       
print("Elapsed: ", time.time() - st) 
print(c, len(all_qs))  
# for train alone:  716 of 9009 <--- using NERper is too narrow on OKVQA
# for train + val: 1110 of 14055 


Elapsed:  85.34579253196716
1110 14055


In [46]:
st = time.time()
all_ents_v4 = get_gen_spans(all_qs , toshow = -1)
print("Elapsed: ", time.time() - st)  #418 last time and led to nothing .. now

Done
Elapsed: 494.3394343852997
Elapsed:  494.3398780822754


In [50]:
#len(all_ents_v4)  #14055
all_ents_nums = [ len(v) for v in all_ents_v4 ]
print(show_stats(all_ents_nums))  #[14055, 4.118]  they all have ents and average is 4 per



[14055, 4.118]

In [62]:
#now make and save out 3 ( orig, 4k, 2p5, with capts ) entity ents
l = len(all_qs[0]["question"]) + 1 #for space after ?
capte = [[v[0],v[1]+l, v[2]+l] for v in all_ents_v4[0]]
comb = all_qs[0]["question"] + " " + all_qs[0]["capt"]
print(all_qs[0], l)
print(orig_ents[0])
orig_ents[0] = [['hairstyle', 12, 21], ['blond', 29, 34]]

{'image_id': 51606, 'question': 'What is the hairstyle of the blond called?', 'question_id': 516065, 'capt': 'Two female tennis players speaking to one another'} 43
[['hairstyle', 12, 21], ['blond', 29, 34]]


In [67]:
#print(orig_ents[0].extend(capte))
#orig_ents[0]
#print(capte)
for v in orig_ents[0]:
    print(v)
    print("====", comb[v[1]:v[2]])

['hairstyle', 12, 21]
==== hairstyle
['blond', 29, 34]
==== blond
['tennis players', 54, 68]
==== tennis players
['speaking', 69, 77]
==== speaking


In [70]:
def add_capt_ents(all_qs, all_ents_v4, orig_ents, k4_ents, k2p5_ents):
    for i in range(len(all_qs)):
        l = len(all_qs[i]["question"]) + 1 #for space after ?
        capte = [[v[0],v[1]+l, v[2]+l] for v in all_ents_v4[i]]
        orig_ents[i].extend(capte)
        k4_ents[i].extend(capte)
        k2p5_ents[i].extend(capte)
        
    return orig_ents, k4_ents, k2p5_ents

norig_ents, nk4_ents, nk2p5_ents = add_capt_ents(all_qs, all_ents_v4, orig_ents, k4_ents, k2p5_ents)

all_ents_nums = [ len(v) for v in norig_ents ]
print("New Orig",show_stats( all_ents_nums))  

all_ents_nums = [ len(v) for v in nk4_ents ]
print("New K4", show_stats(all_ents_nums))

all_ents_nums = [ len(v) for v in nk2p5_ents ]
print("New K2p5", show_stats(all_ents_nums))

New Orig [14055, 6.368]
New K4 [14055, 4.447]
New K2p5 [14055, 4.307]


In [74]:
"""
Orig [13857, 2.25]  --> New Orig [14055, 6.368]
  K4 [4076, 0.329]    --> New K4 [14055, 4.447]
K2p5 [2504, 0.189]  --> New K2p5 [14055, 4.307]
"""

#NOW 1) save out ent sets and 
#and 2) run experiments on each set and on question + caption for OKVQA

#orig_ents = json.load(open(abs_path + "data/okvqa/okvqa_ent_spans_may06_excluded.json"))  
#k4_ents = json.load(open(abs_path +  "data/okvqa/okvqa_ent_spans_sept08_cleaned.json")) 
#k2p5_ents = json.load(open(abs_path +  "data/okvqa/okvqa_ent_spans_sept08_cleaned_filtered.json"))

with open(abs_path + "data/okvqa/okvqa_ent_spans_may06_excluded_with_capts.json","w") as f:
    json.dump(norig_ents, f)
    
with open(abs_path + "data/okvqa/okvqa_ent_spans_sept08_cleaned_with_capts.json","w") as f:
    json.dump(nk4_ents, f)
    
with open(abs_path + "data/okvqa/okvqa_ent_spans_sept08_cleaned_filtered_with_capts.json","w") as f:
    json.dump(nk2p5_ents, f)

In [ ]:
# do 2 above, copy nerper_entity to repo along with InspectResults OKVQA